In [34]:
from pynq import Overlay
ov = Overlay('aivo.bit')

In [35]:
ov.clock_dict

{0: {'divisor0': 5, 'divisor1': 4, 'enable': 1},
 1: {'divisor0': 1, 'divisor1': 1, 'enable': 0},
 2: {'divisor0': 1, 'divisor1': 1, 'enable': 0},
 3: {'divisor0': 1, 'divisor1': 1, 'enable': 0}}

In [36]:
from pynq import MMIO

base_address = 0x43C00000
ctrl_size = 2**10
# imem_size = 2**16
# dmem_size = 2**14
# pmem_size = 2**0

ctrl_mmap = MMIO(base_address, ctrl_size)
imem_size = ctrl_mmap.read(0x318)
dmem_size = ctrl_mmap.read(0x314)
pmem_size = ctrl_mmap.read(0x31C)

# for k in range(0x310, 0x320, 0x4):
#     print(ctrl_mmap.read(k))

    
region_size = max([ctrl_size, imem_size, dmem_size, pmem_size])
imem_mmap = MMIO(base_address + region_size, imem_size)
dmem_mmap = MMIO(base_address + region_size*2, dmem_size)
pmem_mmap = MMIO(base_address + region_size*3, pmem_size)

In [37]:
print("imem_size: {} \n dmem_size: {}".format(imem_size, dmem_size))


imem_size: 65536 
 dmem_size: 16384


In [38]:
i = 0
with open("main.img", mode="r") as file:
    for line in file:
        #print(line)
        #print(len(line))
        line = line.strip().rjust(128, '0') # 128
        #print(line)
        for j in [3,2,1,0]: # 3
            i_word = int(line[(32*j):(32*j+32)], 2)
            imem_mmap.write(i, i_word)
            i += 4

In [39]:
i = 0
with open("main_data.img", mode="r") as file:
    for line in file:
        line = line.strip().rjust(32, '0')
        i_word = int(line, 2)
        if i_word != 0:
           print("{} at {}".format(i_word, i))
        dmem_mmap.write(i, i_word)
        i += 4

65536 at 256
196610 at 260
327684 at 264
458758 at 268
589832 at 272
720906 at 276
851980 at 280
983054 at 284
1114128 at 288
1245202 at 292
1376276 at 296
1507350 at 300
1638424 at 304
1769498 at 308
1900572 at 312
2031646 at 316
131073 at 512
262147 at 516
393221 at 520
524295 at 524
655369 at 528
786443 at 532
917517 at 536
1048591 at 540
1179665 at 544
1310739 at 548
1441813 at 552
1572887 at 556
1703961 at 560
1835035 at 564
1966109 at 568
2097183 at 572
131074 at 768
196610 at 772
327684 at 776
458758 at 780
589832 at 784
720906 at 788
851980 at 792
983054 at 796
1114128 at 800
1245202 at 804
1376276 at 808
1507350 at 812
1638424 at 816
1769498 at 820
1900572 at 824
2031646 at 828
131073 at 1024
262147 at 1028
393221 at 1032
524295 at 1036
655369 at 1040
786443 at 1044
917517 at 1048
1048591 at 1052
1179665 at 1056
1310739 at 1060
1441813 at 1064
1572887 at 1068
1703961 at 1072
1835035 at 1076
1966109 at 1080
29 at 1084
31 at 1280
30 at 1284
29 at 1288
28 at 1292
27 at 1296
26 at

In [40]:
i = 0
with open("main_parameters.img", mode="r") as file:
    for line in file:
        line = line.strip().rjust(32, '0')
        i_word = int(line, 2)
        #if i_word != 0:
        #    print("{} at {}".format(i_word, i))
        pmem_mmap.write(i, i_word)
        i += 4

In [41]:
dmem_mmap.write(5248, 5)

In [42]:
ctrl_mmap.write(0x200, 0x2)

In [43]:
for i in range(0, dmem_size, 4):
    val = dmem_mmap.read(i)
    if val != 0:
        print("b{}, 0x{}, {} at {}".format(bin(val)[2:].zfill(32), hex(val)[2:].zfill(8), val, i))

b00000000000000010000000000000000, 0x00010000, 65536 at 256
b00000000000000110000000000000010, 0x00030002, 196610 at 260
b00000000000001010000000000000100, 0x00050004, 327684 at 264
b00000000000001110000000000000110, 0x00070006, 458758 at 268
b00000000000010010000000000001000, 0x00090008, 589832 at 272
b00000000000010110000000000001010, 0x000b000a, 720906 at 276
b00000000000011010000000000001100, 0x000d000c, 851980 at 280
b00000000000011110000000000001110, 0x000f000e, 983054 at 284
b00000000000100010000000000010000, 0x00110010, 1114128 at 288
b00000000000100110000000000010010, 0x00130012, 1245202 at 292
b00000000000101010000000000010100, 0x00150014, 1376276 at 296
b00000000000101110000000000010110, 0x00170016, 1507350 at 300
b00000000000110010000000000011000, 0x00190018, 1638424 at 304
b00000000000110110000000000011010, 0x001b001a, 1769498 at 308
b00000000000111010000000000011100, 0x001d001c, 1900572 at 312
b00000000000111110000000000011110, 0x001f001e, 2031646 at 316
b0000000000000010

b00000000000000000000000111110000, 0x000001f0, 496 at 15976
b00000000000000011111111111111111, 0x0001ffff, 131071 at 15980
b11111111111111110000000000000000, 0xffff0000, 4294901760 at 15984
b01111111111111111111111111111100, 0x7ffffffc, 2147483644 at 15988
b11000000000000000000000000000011, 0xc0000003, 3221225475 at 15992


In [11]:
from pynq.ps import Clocks

In [12]:
Clocks.fclk0_mhz

50.0